 # Tutorial for modifying behavior in pyfroc

pyfroc has CLI options to modify its behavior. In some cases, these are not enough in your environment. Here are example scenarios.

- The responses are recorded in another file format instead of .nrrd.
- You want to use lesions not approximated as a sphere but in exact 3D volume.

## Base classes to change pyfroc's behavior

- BaseLoader: loading a reference lesion or response
- BaseLesion and BaseResponse: the representation of reference lesion and response
- BaseRater: Devide each reasponse into true or false positive
- BaseWriter: Write a file for further analysis (e.g. xlsx file for RJafroc)

## Make a CsvLoader

In this tutorial, we will define a CsvLoader to read CSV files in which responses are recorded.

To make your own behavior, define a class inheriting BaseLoader and define 

In [ ]:
import csv
import glob
import os
from typing import Sequence

from pyfroc.coords import ScannerCoordinates
from pyfroc.loaders import BaseLoader
from pyfroc.signals import Response


class CsvLoader(BaseLoader):
    # Concrete implementation of the abstract method of BaseLoader
    def read_responses(self, case_dir_path: str) -> Sequence[Response]:
        series_responses: list[Response] = []

        for csv_path in CsvLoader.list_csv_path(case_dir_path):
            series_responses.extend(CsvLoader.read_csv(csv_path))

        return series_responses

    # Definitions of utility methods
    #
    @staticmethod
    def list_csv_path(dir_path: str) -> Sequence[str]:
        return glob.glob(os.path.join(dir_path, "*.csv"))

    @staticmethod
    def read_csv(csv_path: str) -> Sequence[Response]:
        ret: Sequence[Response] = []
        
        with open(csv_path, "r") as f:            
            csv_reader = csv.DictReader(f)
            
            for row in csv_reader:
                x = float(row["x"])
                y = float(row["y"])
                z = float(row["z"])
                coords = ScannerCoordinates(x, y, z)
                
                confidence = float(row["confidence"])
                r = float(row["r"])
                name = row["name"]

                response = Response(coords, confidence, r, name)

                ret.append(response)
            
        return ret

Then, pass an instance of the defined loader class.

In [ ]:
from pyfroc.raters import WithinLesionRater
from pyfroc.writers import RJAFROCWriter, ResponseLesionImagesWriter

experiment_dir_path = "experiment"
xlsx_path = "rjafroc.xlsx"

loader = CsvLoader(experiment_dir_path)
loader.validate_dirs()

rater = WithinLesionRater(loader)

RJAFROCWriter.write(xlsx_path, rater)

# If you want to output the images of the responses in the lesions
dicom_dir_path = "dicom"
ResponseLesionImagesWriter.write(rater, dicom_dir_path, experiment_dir_path)
